In [ ]:
import matplotlib
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

fontpath = '../bigdata anlysis II/src/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
matplotlib.font_manager._rebuild()

Matplotlib is building the font cache; this may take a moment.


# Step1. 탐색: 데이터의 기초 정보 살펴보기

#### * drinks 데이터셋의 기초정보 출력하기

In [ ]:
import pandas as pd
file_path = "data/drinks.csv"
drinks = pd.read_csv(file_path, sep ='\t')
print(drinks.info())
drinks.head(5)

- 책에 주어진 데이터 프레임과 데이터로 현재 데이터프레임이 일치하지 않아  
맞추는 수정작업이 필요하다.

In [ ]:
cols = str(drinks.columns[0])
cols = cols.split(',')
cols

In [ ]:
new_drinks = pd.DataFrame(columns=cols)
for i in range(len(drinks)):
     new_drinks.loc[i] = str(drinks.loc[i][0]).split(",")
new_drinks

In [ ]:
# 데이터프레임 생성 후 한 줄씩 추가하는 방법
df4 = pd.DataFrame(columns=("i1", "i2", "i3"))
for i in range(3):
    df4.loc[i] = [(i + 1) * (n + 1) for n in range(3)]
print(df4); print()
    
# 여러 컬럼을 가진 데이터프레임 한 번에 생성하는 방법
df2 = pd.DataFrame({"c1": [1, 2, 3], "c2": [11, 22, 33], "c3": [111, 222, 333]}, index=["i1", "i2", "i3"])
print(df2)

In [ ]:
drinks = new_drinks
drinks.info()

In [ ]:
# Dtype 바꾸기
# beer_servings, spirit_servings, wine_servings -> int64
# total_litres_of_pure_alcohol  -> float64
cols = ['beer_servings', 'spirit_servings', 'wine_servings']
for col in cols:
    drinks = drinks.astype({col: 'int64'})

col = 'total_litres_of_pure_alcohol'
drinks = drinks.astype({col: 'float'})
drinks.info()

In [ ]:
drinks.describe()

# Step2. 인사이트의 발견: 탐색과 시각화하기

#### * 두 피처간 상관계수 구하기

In [ ]:
#  pearson은 상관 계수를 구하는 계산 방법 중 하나로, 가장 널리 쓰이는 방법
corr  = drinks[['beer_servings','wine_servings']].corr(method='pearson')
print(corr)

#### * 여러 피처간 상관계수 분석하기

In [ ]:
corr = drinks[drinks.columns].corr(method='pearson')
print(corr)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

cols_view = ['beer', 'spirit', 'wine', 'alcohol']
sns.set(font_scale=1.0)
hm = sns.heatmap(corr.values, 
                               cbar = True, 
                               annot = True,
                               square = True,
                               fmt = '.2f',
                               annot_kws = {'size': 15},
                               yticklabels = cols_view,
                               xticklabels = cols_view)

plt.tight_layout()
plt.show()

sns.set(style='whitegrid', context='notebook')
sns.pairplot(drinks[drinks.columns], height = 2.5)
plt.show()

- total_litres_of_pure_alcohol 피처가 대체적으로 모든 피처와 상관관계가 있는 것으로 보아  
특히 beer_servings와 연관성이 많아보인다.

# Step3. 데이터 전처리: 전처리 함수 사용하기

#### * 결측데이터 전처리하기

In [ ]:
#  책에서는 Nan(결측지)으로 되어있지만 지금 자료는 NA으로 되어있다. Other -> OT
# drinks['continent'] = drinks['continent'].fillna('OT')
drinks.replace('NA', 'OT', inplace=True)
drinks.head(10)

#### * 전체 대륙 중에서 OT가 차지하는 비율을 파이차트로 시각화하기

In [ ]:
labels = drinks['continent'].value_counts().index.tolist()
fracs1 = drinks['continent'].value_counts().values.tolist()
explode = (0, 0, 0, 0.25, 0, 0)

plt.pie(fracs1, explode=explode, labels=labels, autopct='%.0f%%', shadow=True)
plt.title('null data to \'OT\'')
plt.show()

#### * agg() 함수를 이용해 대륙별 spirit_servings의 통계적 정보 분석하기

In [ ]:
result = drinks.groupby('continent').spirit_servings.agg(['mean', 'min', 'max', 'sum'])
result.head()

In [ ]:
import numpy as np

n_grps = len(result.index)
means = result['mean'].tolist()
mins = result['min'].tolist()
maxs = result['max'].tolist()
sums = result['sum'].tolist()

index = np.arange(n_grps)
bar_width = 0.1

rt1 = plt.bar(index, means, bar_width, color='r', label='Mean')
rt2 = plt.bar(index + bar_width, mins, bar_width, color='g', label='Min')
rt3 = plt.bar(index + bar_width * 2, maxs, bar_width, color='b', label='Max')
rt4 = plt.bar(index + bar_width * 3, sums, bar_width, color='y', label='Sum')

plt.xticks(index, result.index.tolist())
plt.legend()
plt.show()

#### * 전체 평균보다 많은 알고올을 섭취하는 대륙 구하기

In [ ]:
total_mean = drinks.total_litres_of_pure_alcohol.mean()
continent_mean = drinks.groupby('continent')['total_litres_of_pure_alcohol'].mean()
continent_over_mean = continent_mean[continent_mean >= total_mean]
continent_over_mean

In [ ]:
continents = continent_mean.index.tolist()
continents.append('mean')
x_pos = np.arange(len(continents))
alcohol = continent_mean.tolist()
alcohol.append(total_mean)

bar_list = plt.bar(x_pos, alcohol, align='center', alpha=0.5)
bar_list[-1].set_color('r')
plt.plot([0., 6], [total_mean, total_mean], 'k--')
plt.xticks(x_pos, continents)

plt.ylabel('total_litres_of_pure_alcohol')
plt.title('대륙별 total_litres_of_pure_alcohol')

#### * 평균 beer_servings가 가장 높은 대륙은 어디일까?

In [ ]:
# beer_continent = drinks.groupby('continent')['beer_servings'].mean().idxmax()
beer_continent = drinks.groupby('continent')['beer_servings'].mean()
print(beer_continent)
beer_continent.idxmax()

In [ ]:
beer_sum = drinks.groupby('continent')['beer_servings'].sum() #평균이 아닌 합을 구함
continents = beer_sum.index.tolist()
y_pos = np.arange(len(continents))
alcohol = beer_sum.tolist()

bar_list = plt.bar(y_pos, alcohol, align='center', alpha=0.5)
bar_list[continents.index('EU')].set_color('r')
plt.xticks(y_pos, continents)
plt.ylabel('beer_serving')
plt.title('대륙별 beer_serving')

plt.show()

# Step4. 탐색적 분석: 스무고개로 개념적 탐색 분석하기 

#### * 아프리카와 유럽 간의 맥주 소비량 차이 검정하기

#### * '대한민국은 얼마나 술을 독하게 마시는 나라일까' 탐색하기

#### * 국가별 순위 정보를 시각화하기